In [1]:
# Import Packages
from port_ctrl1 import Rigol_DCPort
from CAENDesktopHighVoltagePowerSupply1 import CAENDesktopHighVoltagePowerSupply, OneCAENChannel
import pyvisa as visa
import time
import numpy as np
import serial
import serial.tools.list_ports 
from matplotlib import pyplot as plt
from DC_Scan_class import *
from random import randint
from UI_Main_Window_Object import Ui_MainWindow
from PlotThread import PlotThread
from DCThread import DCThread

import sys
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt, QTimer, QThread, QMutex
from PyQt5.QtWidgets import *
from PyQt5 import QtCore, QtGui, QtWidgets
import pyqtgraph as pg

In [2]:
def get_com_list():
    Com_List = []
    plist = list(serial.tools.list_ports.comports())
    if len(plist) > 0:
        for i in range(len(plist)):
            Com_List.append(list(plist[i])[0])
    return Com_List

In [3]:
#Logic
class MyWindow(QMainWindow,Ui_MainWindow):
    def __init__(self, parent=None):
        global DC
        super(MyWindow, self).__init__(parent)
        self.setupUi(self)
        self.data0 = None
        self.data1 = []
        self.complete=False
        self.count = 0
        self.thread=None
        self.control_thread=None
        self.time=None
        self.DC_mon=None
        self.RF_mon=None
        self.setWindowIcon(QIcon('serialscope.ico'))
        self.sendbutton.setToolTip('Click to Open the port')
        self.sendbutton.clicked.connect(self.open_com)
        self.sendcom.activated['QString'].connect(self.port_changed)
        self.sendbot.activated['QString'].connect(self.baud_changed)
        self.confbutton.clicked.connect(self.config)
        self.cancelbutton.clicked.connect(self.cancel)
        self.timer = QTimer()
        self.timer.timeout.connect(self.Timer)
        self.thread=PlotThread(self.plotwindow, DC)
        
    def show_dialog(self,str,title=""):
        # Create QDialog object
        dialog = QDialog()
        layout = QVBoxLayout(dialog)
        label_layout = QHBoxLayout()
        button_layout = QHBoxLayout()
        
        # Create QLabel and QPushButton in the dialog
        lb = QLabel(str, dialog)
        btn = QPushButton('ok', dialog)
        
        lb.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        btn.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        
        label_layout.addWidget(lb)
        button_layout.addWidget(btn)
        label_layout.setAlignment(Qt.AlignCenter)
        
        layout.addLayout(label_layout)
        layout.addLayout(button_layout)
        btn.clicked.connect(dialog.accept)
    
        # Set dialog title
        dialog.setWindowTitle(title)
        
        # Set the window modality and adjust the size to fit the contents
        dialog.setWindowModality(Qt.NonModal)
        dialog.adjustSize()
        
        # Show the dialog
        dialog.exec_()
    
    #######################
    def open_com(self): # Connecting HV DC Supply
        global ser, serialPort, baudRate, com_list
        com_list = get_com_list() # Get available Ports  
        DC.working_message="Checking Connection..."
        self.messagewindow.append(DC.working_message)
        if com_list != []:  
            if serialPort != None and self.sendcom.currentText() != "":# Port is not None and Connect button is clicked
                ser.port = serialPort
                ser.baudrate = baudRate
                if self.sendbutton.text() == 'Connect':
                    ser.open() # Cannot open an open port
                    #self.timer.start(timer_value) #Start Timer for message display
                    DC.working_message="Checking Connection Complete"
                    self.messagewindow.append(DC.working_message)
                    self.sendcom.setEnabled(False)
                    self.sendbot.setEnabled(False)
                    self.sendbutton.setText("Disconnect")
                    self.sendbutton.setToolTip('Click to Close Port')
                    print('Port Connect')
                else:
                    ser.close()  # Close the port
                    self.sendcom.setEnabled(True)
                    self.sendbot.setEnabled(True)
                    self.sendbutton.setText("Connect")
                    self.sendbutton.setToolTip('Click to Open Port')
                    print('Port Disconnect')
            elif serialPort != None and self.sendcom.currentText() == "":
                myWin.show_dialog(str='Please Choose a Port', title='Warning：No Port Chosen')
            else:  # Fixing
                myWin.port_combo.clear()
                for i in range(len(com_list)):  # Add available Ports to combo box
                    myWin.sendcom.addItem(com_list[i])
                serialPort = com_list[0]  # Setting the first available port by default
        else:  # Show warning dialog
            serialPort = None
            myWin.sendcom.clear()
            myWin.show_dialog(str='No Port Detected', title='Warning：Port Unfound')
            
    def Check_scan(self):
        global DC
        # Mode selection check
        if not self.auto.isChecked() and not self.custom.isChecked():
            self.show_dialog(str='Please Choose a Mode',title="Warning: No Mode Selected")
            DC.error_message="No Mode Selected"
            self.messagewindow.append(DC.error_message)
            return False
        else:
            DC.working_message="Mode Check Complete\n" + "Start RF and DC Check"
            self.messagewindow.append(DC.working_message)
        
        # Starting and Ending value Check
        try:
            DC.Arange = [float(self.Aminputdata.text()), float(self.AMinputdata.text())]
            DC.Hrange = [float(self.Qminputdata.text()), float(self.QMinputdata.text())]
            DC.working_message="RF and DC Check Complete\n" + "Start Channel Check"
            self.messagewindow.append(DC.working_message)
        except:
            self.show_dialog(str='Amplitude or DC unspecified',title="Warning: Invalid Input")
            DC.error_message="RF or DC Unspecified"
            self.messagewindow.append(DC.error_message)
            return False
        
        # Check if channel has been specified
        try:
            int(self.AChannel.currentText()[-1])
            int(self.QChannel.currentText()[-1])
            int(self.QChannel1.currentText()[-1])
            DC.Achannel = self.AChannel.currentText()
            DC.Hchannel = self.QChannel.currentText()
            DC.Hchannel1 = self.QChannel1.currentText()
            DC.working_message="Channel Check Complete\n" + "Start Mode Parameter Check"
            self.messagewindow.append(DC.working_message)
        except:
            self.show_dialog(str='Channels unspecified',title="Warning: Invalid Input")
            DC.error_message="Channel Unspecified"
            self.messagewindow.append(DC.error_message)
            return False
        
        # Detailed checking for different mode
        if self.auto.isChecked():
            DC.mode=0
            try:
                DC.sampling=int(self.Sample.text())
                DC.sampling_validity() # Setting curr_q, curr_a, sampling
                DC.working_message="Parameter Check Complete\n" + "Start Duration Check..."
                self.messagewindow.append(DC.working_message)
            except:    
                self.show_dialog(str='Number of Samples Unspecified',title="Warning: Invalid Input")
                return False
        elif self.custom.isChecked():
            DC.mode=1
            try:
                DC.Astep=float(self.Astep.text())
                DC.Hstep=float(self.Hstep.text())
                DC.step_validity()
                DC.sampling_validity() # Setting curr_q, curr_a, sampling
                DC.working_message="Parameter Check Complete\n" + "Start Duration Check..."
                self.messagewindow.append(DC.working_message)
            except:
                try:
                    DC.Astep=map(float,self.Astep.text().split(','))
                    DC.Hstep=map(float,self.Hstep.text().split(','))
                    
                    if len(DC.Astep) != len(DC.Hstep):
                        self.show_dialog(str='Number of Steps Mismatch',title="Warning: Invalid Input")
                        DC.error_message="Step Mismatch"
                        self.messagewindow.append(DC.error_message)
                        return False
                except:
                    self.show_dialog(str='Error in Steps Input',title="Warning: Invalid Input")
                    DC.error_message="Invalid Step Input"
                    self.messagewindow.append(DC.error_message)
                    return False
            
        if self.auto.isChecked():   
            try:
                DC.intv = int(self.Duration.text()) # Check if input is integer
                DC.duration_check()
                DC.working_message="Duration Check Complete"
                self.messagewindow.append(DC.working_message)
            except:
                try:
                    if self.Duration.text()=="": # No input, run default mode
                        DC.intv=None
                        DC.duration_check()
                    else:
                        Duration = self.Duration.text().split(',') # input list
                        if Duration[-1] == "": # exclude the case with ending comma
                            Duration.pop()
                        for i in range(len(Duration)):
                            Duration[i] = int(Duration[i])
                        DC.intv = Duration
                        DC.duration_check()
                except:
                    self.show_dialog(str='Number of Duration Mismatch',title="Warning: Invalid Input")
                    DC.error_message="Duration Mismatch"
                    self.messagewindow.append(DC.error_message)
                    return False
        return True
            
    def cancel(self):
        ser.close()
        self.timer.stop()
        self.sendcom.setEnabled(True)
        self.sendbot.setEnabled(True)
        self.sendbutton.setText("Connect")
        self.sendbutton.setToolTip('Click to Open Port')
        print('Port Disconnect')
        DC.working_message="Current Program Canceled"
        self.messagewindow.append(DC.working_message)
        self.confbutton.setEnabled(True)
        self.cancelbutton.setEnabled(False)
        self.auto.setEnabled(True)
        self.custom.setEnabled(True)
        self.auto.setChecked(False)
        self.custom.setChecked(False)
        self.AChannel.setEnabled(False)
        self.QChannel.setEnabled(False)
        self.QChannel1.setEnabled(False)
        self.Aminputdata.setReadOnly(True)
        self.AMinputdata.setReadOnly(True)
        self.Qminputdata.setReadOnly(True)
        self.QMinputdata.setReadOnly(True)
        self.Duration.setReadOnly(True)
        self.Astep.setReadOnly(True)
        self.Hstep.setReadOnly(True)
        self.Sample.setReadOnly(True)
        DC.cancel_configuration()
        self.thread.terminate()
        self.control_thread.terminate()
    
    def port_changed(self, text):
        global ser, serialPort
        serialPort = text
        print('Port：' + serialPort)
        ser.port = serialPort
        
    def baud_changed(self, text):
        global ser, baudRate
        baudRate = int(text)
        print(baudRate)
        ser.baudrate=baudRate
    
    def config(self):
        global serialPort, DC
        if serialPort != None :
            if self.sendbutton.text() == 'Disconnect'and self.sendcom.currentText() != "":
                DC.working_message="Checking Mode Configuration..."
                self.messagewindow.append(DC.working_message)
                # Check if inputs are valid
                self.confbutton.setEnabled(False)
                self.cancelbutton.setEnabled(True)
                result = self.Check_scan()
                if result == False:
                    DC.error_message=("Error occurs, configure cancelled.")
                    self.messagewindow.append(DC.error_message)
                    self.cancel()
                else:
                    DC.working_message="Configuration Complete.\n" + "Initializing..."
                    self.messagewindow.append(DC.working_message)
                    self.disable_parameter()
                    if self.auto.isChecked() or self.custom.isChecked():
                        self.DC_Scan()
                        if self.complete:
                            try:
                                DC.working_message="Program Complete."
                                self.messagewindow.append(DC.working_message)
                                self.enable_parameter()
                                self.Astep.setReadOnly(True)
                                self.Hstep.setReadOnly(True)
                            except:
                                try:
                                    DC.error_message="Error Ocurrs, Scanning Canceled."
                                    self.messagewindow.append(DC.error_message)
                                    self.cancel()
                                except:
                                    self.show_dialog(str='DC Devices Not Found',title="Warning: No DC Devices")
            elif (self.sendbutton.text() == 'Connect' or self.sendbutton.text() == 'Port Conn')and self.sendcom.currentText() != "":
                self.show_dialog(str='Please Connect to the Port',title="Warning: No Connection")
            else:
                self.show_dialog(str='Please Choose a Port',title="Warning: No Port Selected")
        else:
            self.show_dialog(str='No Port Found',title="Warning: No Port Selected")
    
    def Timer(self):
        self.count+=1
    
    def DC_Scan(self):
        global DC, ser
        
        # self.control_thread=DCThread(DC, self.thread)
        DC.RF_channel("ON")
        DC.DC_channel(DC.Hchannel,"ON")
        DC.DC_channel(DC.Hchannel1,"ON")
        
        # Heat up the HV
        DC.RF_set(DC.curr_a[0])
        DC.DC_set(DC.curr_q[0], DC.Hchannel)
        self.count=0
        self.timer.start(1000)
        while self.count < 30:
            QApplication.processEvents()
        self.timer.stop()
        
        DC.working_message="Initialization Complete.\n" + "Start Scanning..."
        self.messagewindow.append(DC.working_message)
        
        self.thread.start()
        self.control_thread.start()
        
        try:
            for i in range(DC.sampling):
                DC.RF_set(DC.curr_a[i])
                DC.DC_set(DC.curr_q[i], DC.Hchannel)
                #self.thread.update_data1(DC.curr_a[i], DC.curr_q[i])
                self.count=0
                self.timer.start(1000) # Timer to pause the main thread
                while self.count < DC.interval[i]:
                    QApplication.processEvents()
                self.timer.stop()
            
        except Exception as e:
            print(f"An error occurred: {e}")
        
        finally:
            self.thread.terminate()
            DC.working_message="Scanning Complete.\n" + "Turning Off Output..."
            self.messagewindow.append(DC.working_message)
            
            # Close all output after scanning
            self.control_thread.stop() # Turn off HV output
            DC.working_message="Output Turned Off.\n" + "Configuration Clear.\n" + "Terminating the Program..."
            self.messagewindow.append(DC.working_message)
            
            timeout = 5  # Define an appropriate timeout
            start_time = time.time()
    
            while self.thread.isRunning() or self.control_thread.isRunning():
                if time.time() - start_time > timeout:
                    self.thread.terminate()
                    #self.control_thread.terminate()
                    break  # Break the loop after a certain timeout
    
                # Allow the GUI to remain responsive
                QApplication.processEvents()
            
            ser.close()
            self.complete=True

<ipython-input-3-900f48f1c17e>:2: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class MyWindow(QMainWindow,Ui_MainWindow):


In [4]:
####################################################
#Gloabal Variables and Initialization
ser=serial.Serial(timeout=1)
DC = DC_Scan(ser)
app = QApplication(sys.argv)
myWin = MyWindow()
myWin.show()
#myWin.Port_send()
baudRate = 9600  # baudrate
timer_value=1
#################################################
com_list=get_com_list() #obtain port list

if com_list!=[]:
    for i in range(len(com_list)):
        myWin.sendcom.addItem(com_list[i])
    serialPort = com_list[0]
else:
    serialPort = None
    myWin.show_dialog(str='Please Open the Port')

sys.exit(app.exec_())

VisaIOError('VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.')


SystemExit: 0

C:\Users\15421\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
